In [1]:
import pandas as pd
import os

In [2]:
# This function turns xy coordinate files into xyz files
def xy2xyz(code, filepath):
    
    # Turn the text file into a Pandas dataframe
    with open(filepath + code + '_building_coords.txt') as file: coords = pd.read_csv(file, sep=',', names=['Area', 'x', 'y'])
    
    # Drop the class column from the frame
    coords = coords.drop(['Area'], axis = 1)

    # Create the normalized dataframe
    normalized_data = pd.DataFrame()
    measure = [[1, 0], [1, 0]]
    index = 0
    abs_max = 0

    # Find the normalization factor for both x and y
    for letter in ['x', 'y']:
        localMax = coords[letter].max()
        localMin = coords[letter].min()
        abs_local = max(abs(localMax), abs(localMin))

        if abs_local > abs_max:
            abs_max = abs_local

    for letter in ['x', 'y']:
        normalized_data[letter] = coords[letter].divide(abs_max)
        mean = normalized_data[letter].mean()
        normalized_data[letter] = normalized_data[letter].subtract(mean)
        normalized_data[letter] = normalized_data[letter].multiply(10000)

        for scale in range(len(measure[index])): 
            num = measure[index][scale]
            num = num/10000 
            num = num + mean  
            num = num * abs_max  
            measure[index][scale] = num

        index += 1

    scale = ((((measure[0][0] - measure[0][1])**2 + (measure[1][0] - measure[1][1])**2)/2)**(1/2))

    # The multiplication factor here comes from the amount of feet in one degree of latitude. It's a rough measure
    line = 'One degree of Ovito coords is about equal to ' + str(int(scale*364000)) + ' feet!'
    
    # Add in the extra columns and rows necessary in an xyz file
    normalized_data.insert(0, 'test', 'H')
    normalized_data[''] = 0
    normalized_data.loc[-1] = ['', '', '', '']
    normalized_data.index = normalized_data.index + 1
    normalized_data = normalized_data.sort_index()
    r, c = normalized_data.shape

    # Setting one of the 'z' values as 0.01 to prevent degeneracy (lol)
    normalized_data.columns = ['1', '2', '3', '4']
    normalized_data['4'][1] = 0.01
    normalized_data.columns = [r - 1, '', '', '']
    
    # Found here: https://stackoverflow.com/questions/41428539/data-frame-to-file-txt-python/41428596
    normalized_data.to_csv(path_or_buf=filepath + code + '_building_coords_reformatted.txt', index=False, header=True, sep='\t', mode='a')

    # Write the scale to a text file
    with open(filepath + code + '_scale.txt', 'w+') as file: file.write(str(line))

In [4]:
path = 'E:/Old Downloads Folder/Research Stuff/Jupyter Notebooks/Coords Folder/Weather Station Coords/Las Vegas/'

xy2xyz('US-LV', path)

# files = os.listdir(path)
# ids = []

# for file in files:
#     num = file[:file.find('_')]
#     if num not in ids:
#         ids.append(num)
        
# for i in ids:
#     xy2xyz(i, path)